In [1]:
import torch
from torch.nn.functional import pad

In [3]:
# Create goal
B = torch.tensor([0, 1, 0, 1, 0])
radB = len(B)//2
# Create coordinate tensor

z = torch.tensor([-0.4, 2.1], requires_grad=True)

optim = torch.optim.Adam([z], lr=1e-1)

In [23]:
rad = max(z.detach().abs().max().ceil().long(), radB)
wc = torch.zeros(2*rad+1)
wf = torch.zeros((2, 2*rad+1))
# Map coordinates to weight matrix
zm = z + rad
wc[zm.ceil().long()] =  1.
wf[0:2, zm.floor().long()] =  1.

In [24]:
wf

tensor([[0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 1., 0.]])

In [12]:
F = (torch.outer(-(zm-zm.floor()).pow(2)+1, wf).mean(0)+torch.outer(-(zm-zm.ceil()).pow(2)+1, wc).mean(0))/1


In [22]:
torch.outer(-(zm-zm.floor()).pow(2)+1, wf)

tensor([[0.0000, 0.0000, 0.6400, 0.0000, 0.0000, 0.6400, 0.0000],
        [0.0000, 0.0000, 0.9900, 0.0000, 0.0000, 0.9900, 0.0000]],
       grad_fn=<MulBackward0>)

In [ ]:
raddiff = (len(F) -  len(B))//2
Fbigger = len(F) > len(B)
if Fbigger:
  #print('Rad diff: ', raddiff)
  Bpad = pad(B, (raddiff,raddiff))
else:
  Bpad = B

In [ ]:
l = ((Bpad-F)**2).sum()
optim.zero_grad()
l.backward()
optim.step()
losses.append(l.item())